In [1]:
import logging
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

# Disable INFO and WARNING messages
logging.getLogger().setLevel(logging.ERROR)

# Specifically for LangChain
logging.getLogger("langchain").setLevel(logging.ERROR)

from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import vizro.plotly.express as px


# world food production 1961-2021
# df1 = pd.read_csv("/Users/lingyi_zhang/vizx/os/world data/world food production.csv")
# print(df1.head(1))



In [2]:
#%pip install Pillow

import base64
from PIL import Image
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from langchain.schema import HumanMessage

from vizro_ai._llm_models import _get_llm_model
model = _get_llm_model(model="gpt-4o")

def load_image(image_path):
    """
    Load an image from a local file path, convert it to base64,
    and return as a numpy array and base64 string.
    """
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")
    
    img = Image.open(image_path)
    return np.array(img), image_data

def get_image_data(**kwargs):
    """
    Load images and return a list of their base64-encoded data.
    
    :param kwargs: Should contain 'image_paths' key with a list of file paths
    :return: List of base64-encoded image data
    """
    image_paths = kwargs.get('image_paths', [])
    return [load_image(path)[1] for path in image_paths]

def create_image_subplot(fig, img_array, row, col, title=None):
    """Add an image as a subplot to the figure."""
    fig.add_trace(
        go.Image(z=img_array),
        row=row, col=col
    )
    if title:
        fig.update_xaxes(title_text=title, row=row, col=col)

def display_images(image_paths, titles=None, ncols=2):
    """Display multiple images in a grid layout."""
    n_images = len(image_paths)
    nrows = (n_images - 1) // ncols + 1

    fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=titles)

    for i, path in enumerate(image_paths):
        img_array, _ = load_image(path)
        row = i // ncols + 1
        col = i % ncols + 1
        create_image_subplot(fig, img_array, row, col)

    fig.update_layout(
        showlegend=False,
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False)
    )

    fig.show()

def construct_message(images, question):
    """
    Construct a HumanMessage with a dynamic number of images.
    
    :param images: List of base64-encoded image data
    :param question: The question to ask about the images
    :return: HumanMessage object
    """
    content = [{"type": "text", "text": question}]
    for img_data in images:
        content.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{img_data}"}
        })
    return HumanMessage(content=content)

# # Example usage
# image_paths = [
#     "/Users/lingyi_zhang/vizx/os/world data/page1.png",
#     "/Users/lingyi_zhang/vizx/os/world data/page2.png",

# ]
# titles = ["First Image", "Second Image"]

# Display images
# display_images(image_paths, titles)
question = """
You are a front-end developer with expertise in Plotly, Dash, and the visualization library named Vizro.
1. Describe how many dashboard pages do you see.
2. Describe what do you see in each page with following aspects:

<Vizro Components>
you might see `Card (which is a container holding text)`, `Graph`, or `AgGrid`.
For `Graph`, Describe the chart type and describe what you see for x-axis, y-axis, and legend. 

<Vizro Filter>
you might see filters using different selectors, including `Dropdown`, `Checklist`, `RadioItems`, `RangeSlider`, `Slider`, `DatePicker`.

<Vizro Layout>
Only describe the positions of all Vizro Components (`Card`, `Graph`, `AgGrid`). 
When describing layout, AVOID the positions of `Filter`.

Important, if there are any text you see, accurately describe the original text. 
"""

In [3]:
# Example usage
image_paths = [
    "/Users/lingyi_zhang/vizx/os/world data/page1.png",
    "/Users/lingyi_zhang/vizx/os/world data/page2.png",
]

# Get image data
images = get_image_data(image_paths=image_paths)

# Construct message
message = construct_message(images, question)

response = model.invoke([message])
dashboard_spec = response.content
print(response.content)

### Dashboard Pages
There are two dashboard pages.

### Page 1: World food expenditure

#### Vizro Components
1. **Graph (Left)**
   - **Chart Type**: Box Plot
   - **X-axis**: Year
   - **Y-axis**: Food as share of total consumer expenditure
   - **Legend**: 2017 (black), 2018 (red), 2019 (purple), and more years indicated by ellipsis (...)
   
2. **Graph (Right)**
   - **Chart Type**: Histogram
   - **X-axis**: Food as share of total consumer expenditure
   - **Y-axis**: Frequency
   - **Legend**: 2017 (black), 2018 (red), 2019 (purple), and more years indicated by ellipsis (...)

#### Vizro Filter
1. **Dropdown**: Entity
2. **Checklist**: Year (2017, 2018, 2019, 2020, 2021)

#### Vizro Layout
- The two `Graph` components are positioned side by side, with the box plot on the left and the histogram on the right.

### Page 2: World GDP and life expectancy

#### Vizro Components
1. **Card**
   - **Text**: 
     - "Gapminder dataset:"
     - "The Gapminder dataset provides historical dat

# Data sources

In [4]:
# 2017-2021
df2 = pd.read_csv("/Users/lingyi_zhang/vizx/os/world data/3- hare-of-consumer-expenditure-spent-on-food.csv")

# 1952-2007
df3 = px.data.gapminder()

# VizroAI

In [5]:
from vizro import Vizro
from vizro_ai import VizroAI

vizro_ai = VizroAI(model="gpt-4o")

In [6]:
Vizro._reset()
dashboard = vizro_ai.dashboard([df2, df3], dashboard_spec)

Vizro().build(dashboard).run(port=6050)

Store df info:   0%|          | 0/2 [00:00<?, ?it/s]

df_name: food_expenditure
df_name: gapminder


Generate dashboard plan:   0%|          | 0/2 [00:00<?, ?it/s]

Building page: World GDP and life expectancy:   0%|          | 0/5 [00:00<?, ?it/s]

Building page: World food expenditure:   0%|          | 0/5 [00:00<?, ?it/s]

Currently Building ... [Page] <World GDP and life expectancy> components:   0%|          | 0/2 [00:00<?, ?it/s…

Currently Building ... [Page] <World food expenditure> components:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/lingyi_zhang/vizx/os/vizro/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Currently Building ... [Page] <World GDP and life expectancy> controls:   0%|          | 0/1 [00:00<?, ?it/s]

Currently Building ... [Page] <World food expenditure> controls:   0%|          | 0/2 [00:00<?, ?it/s]